In [34]:
from pathlib import Path

from cleantext import clean
import dataset
import requests

In [5]:
all_data = []
all_loc = []
all_src = []

for p in Path('data').glob('*'):
    print(p)
    try:
        db = dataset.connect('sqlite:///'  + str(p))
        tab_data = db['incidents'].all()
        tab_loc = db['locations'].all()
        tab_src = db['sources'].all()
        all_data += tab_data
        all_loc += tab_loc
        all_src += tab_src
    except:
        print('error')

data/data_2.db
data/data_1.db
data/data_0.db


In [6]:
db = dataset.connect('sqlite:///rechtegewalt.db')

tab_data = db['incidents']
tab_data.insert_many(all_data)
tab_loc = db['locations']
tab_loc.insert_many(all_loc)
tab_src = db['sources']
tab_src.insert_many(all_src)

tab_data.create_index(['identifier'])
tab_loc.create_index(['identifier'])
tab_src.create_index(['identifier'])

In [49]:
# clean location text because there were still some errors
for x in tab_loc.all():
    x['subdivisions'] = clean(x['subdivisions'], lang='de', lower=False)
    tab_loc.update(x, ['id'])

In [51]:
subs = list(tab_loc.distinct('subdivisions'))
subs = [x['subdivisions'] for x in subs]

In [52]:
auth = tuple(Path('secrets.txt').read_text().split()[1:])

In [53]:
for s in subs:
#     print(s)
    r = requests.get(f'https://geocode.app.vis.one/?q={s}', auth=auth)
#     print(r)
    if r.ok:
        data = r.json()
        data['subdivisions'] = s
        tab_loc.update(data, ['subdivisions'])

"Freies Netz" mit genehmigtem Stand auf dem Stadtfest in Geithain, LK Leipzig, Sachsen, Deutschland
<Response [200]>
(Geithain), LK Leipzig, Sachsen, Deutschland
<Response [200]>
, Sachsen-Anhalt, Deutschland
<Response [200]>
A4, LK Bautzen, Sachsen, Deutschland
<Response [200]>
Ahrensfelde, Barnim, Brandenburg, Deutschland
<Response [200]>
Aken, Kreis Anhalt-Bitterfeld, Sachsen-Anhalt, Deutschland
<Response [200]>
Aken, Landkreis Köthen, Sachsen-Anhalt, Deutschland
<Response [200]>
Aken, Sachsen-Anhalt, Deutschland
<Response [200]>
Aken/Elbe, Sachsen-Anhalt, Deutschland
<Response [200]>
Alleringersleben, Börde, Sachsen-Anhalt, Deutschland
<Response [200]>
Altchemnitz, Stadt Chemnitz, Sachsen, Deutschland
<Response [200]>
Altdöbern, Oberspreewald-Lausitz, Brandenburg, Deutschland
<Response [200]>
Altenberg OT Geising, Sächsische Schweiz-Osterzgebirge, Sachsen, Deutschland
<Response [200]>
Altenberg, Sächsische Schweiz-Osterzgebirge, Sachsen, Deutschland
<Response [200]>
Altendorf, Stad

In [56]:
# Not really sure whether a seperate table for location is needed. It was introduced because in some cases, multiple locations are associated with incident.
for x in tab_data.all():
    row_loc = tab_loc.find_one(identifier=x['identifier'])
    merged = {**row_loc, **x}
    tab_data.update(merged, ['id'])